# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
# Библиотека pandas-profiling не всегда устанавливается стабильно, поэтому принудительно задать стабильные версии
!pip install pandas_profiling==1.4.1 -q
!pip install pandas==0.25.3 -q

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split # Разделение выборок
from sklearn.preprocessing import StandardScaler # Масштабирование

from sklearn.linear_model import LinearRegression # Линейная регрессия

from pandas_profiling import ProfileReport

from sklearn.metrics import mean_squared_error # Вычисление RMSE

from math import sqrt # Корень квадратный

import random

ModuleNotFoundError: No module named 'pandas_profiling'

## Загрузка и подготовка данных

In [3]:
# Загружаем файлы 
df0 = pd.read_csv('/datasets/geo_data_0.csv', sep=',')
df1 = pd.read_csv('/datasets/geo_data_1.csv', sep=',')
df2 = pd.read_csv('/datasets/geo_data_2.csv', sep=',')

# Смотрим общую информацию о таблице
print(df0.info(), df1.info(), df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

In [4]:
# Изучим данные по первому региону
profile = ProfileReport(df0)
profile

/home/jovyan/.local/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,40.0 B
Numeric,4
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [5]:
# Изучим данные по второму региону
profile = ProfileReport(df1)
profile

/home/jovyan/.local/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,40.0 B
Numeric,3
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [6]:
# Изучим данные по третьему региону
profile = ProfileReport(df2)
profile

/home/jovyan/.local/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,40.0 B
Numeric,4
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


### Выводы

По результату анализа представленных датасетов можно сделать следующие выводы:
- Пропусков, некорректных наименований и типов данных не обнаружено
- Стоит провести масштабирование признаков

## Обучение и проверка модели

In [7]:
# Создадим функция для подготовки данных, обучения и проверки модели
def preparation_creature (df):
    
    # Создание признаков
    features = df.drop(['product', 'id'], axis=1)

    # Создание целевого признака
    target = df['product']

    # отделим 25% данных для валидационной выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    # Сформируем таблицу для проверки разделения выборки (использовал при отладке)
#     data = {'Размер выборки':[len(features_train), len(features_valid)], 'В % от общего количества':[len(features_train)/len(features)*100, len(features_valid)/len(features)*100] } 
#     selections = pd.DataFrame(data, index =['Тренировочная выборка', 'Валидационная выборка']) 
#     display(selections)
    
    scaler = StandardScaler()

    # Обучаем модель на тренировочной выборке
    scaler.fit(features_train)

    # Масштабируем тренировочную и валидационные выборки
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid) 
    
    model = LinearRegression() 

    # Обучаем модель на тренировочной выборке
    model.fit(features_train, target_train) 

    # предсказания модели на валидационной выборке
    predictions_valid = model.predict(features_valid) 
    # Расчет RMSE
    result = sqrt(mean_squared_error(target_valid, predictions_valid))
    print('RMSE модели:', result)

    print('Средний запас предсказанного сырья:', target.mean())
    
    predictions_valid = pd.Series(predictions_valid, index=target_valid.index)
   
    return target_valid, predictions_valid

In [8]:
# Применим функцию для первого датафрейма
target_valid_0, predictions_valid_0 = preparation_creature (df0)

RMSE модели: 37.5794217150813
Средний запас предсказанного сырья: 92.50000000000001


In [9]:
# Применим функцию для второго датафрейма
target_valid_1, predictions_valid_1 = preparation_creature (df1)

RMSE модели: 0.8930992867756158
Средний запас предсказанного сырья: 68.82500000000002


In [10]:
# Применим функцию для третьего датафрейма
target_valid_2, predictions_valid_2 = preparation_creature (df2)

RMSE модели: 40.02970873393434
Средний запас предсказанного сырья: 95.00000000000004


### Выводы

После проведения масштабирования признаков и обучения моделей, на основе показателя RMSE можно сделать вывод, что наилучшим образом отработала модель по второму региону

## Подготовка к расчёту прибыли

In [11]:
# Бюджет на разработку скважин в регионе, руб
BUDGET = 10000000000

# Доход с тысячи баррелей составляет, руб
INCOME = 450000

# Количество скважин для разработки, шт
NUMBER_OF_WELLS = 200

# Количествто точек для разведки региона
EXPLORATION = 500

# 2,5% квантиль
KVANT25 = 0.025

In [12]:
# Расчитаем средний объем скважин, необходимый для достижения точки безубыточности
point = BUDGET/NUMBER_OF_WELLS/INCOME
print('Для того чтобы достичь точки безубыточности наполненность скважин в среднем должна быть не менее:', point)

Для того чтобы достичь точки безубыточности наполненность скважин в среднем должна быть не менее: 111.11111111111111


In [13]:
   # Сформируем таблицу для сравнения средних показателей с точкой безубыточности
data = {'Среднее значение запасов':[df0['product'].mean(), df1['product'].mean(), df2['product'].mean()], 'Разница между средним значением запасов и точкой безубыточности':[df0['product'].mean()-point, df1['product'].mean()-point, df2['product'].mean()-point] } 
selections = pd.DataFrame(data, index =['Первый регион', 'Второй регион', 'Третий регион']) 
display(selections)


,Среднее значение запасов,Разница между средним значением запасов и точкой безубыточности
Первый регион,92.500,-18.611111
Второй регион,68.825,-42.286111
Третий регион,95.000,-16.111111


### Выводы

Исходя из полученных расчетов, а именно - точка безубеточности в объеме 111 тыс.баррелей и полученные средние показатели по исследуемым регионам, можно сделать вывод, что на данном этапе исследования, основанном на получении осредненных показателей добычи, разработка регионов нерентабельна.

## Расчёт прибыли и рисков 

In [14]:
# Напишем функцию подсчета прибыли
def revenue(target, predictions, count):
   
    predictions = pd.Series(predictions, index=target.index)
    
    # Выстраиваем предсказания по убыванию
    probs_sorted = predictions.sort_values(ascending=False)
    
    # Затем, чтобы выбрать ответы, возьмем их индексы: 
    income_int = target[probs_sorted.index][:count].sum()*INCOME
   
    revenue_sum = income_int-BUDGET
    return revenue_sum 

In [15]:
   # Сформируем таблицу для подсчтеа прибыли к прогнозу модели по регионам
data = {'Прибыль, млрд.руб.':[revenue(target_valid_0, predictions_valid_0, NUMBER_OF_WELLS), revenue(target_valid_1, predictions_valid_1, NUMBER_OF_WELLS), revenue(target_valid_2, predictions_valid_2, NUMBER_OF_WELLS)] } 
selections = pd.DataFrame(data, index =['Первый регион', 'Второй регион', 'Третий регион']) 
display(selections)

,"Прибыль, млрд.руб."
Первый регион,3.320826e+09
Второй регион,2.415087e+09
Третий регион,2.710350e+09


In [16]:
# Применим технику Bootstrap с 1000 выборок

state = np.random.RandomState(12345)

# Создадим функцию для проведения Bootstrap
def analysis (target, predictions): 
    values = []
    for i in range(1000):
        # Проводим бутсреп на 500 значений
        target_subsample = target.sample(n=EXPLORATION, random_state=state, replace=True)
        
        # Отберем значения предсказаний по выборке бутсрепа
        probs_subsample = predictions[target_subsample.index]
        
        values.append(revenue(target_subsample, probs_subsample, NUMBER_OF_WELLS))# < напишите код здесь>
    
    values = pd.Series(values)/1e6
    # Вычислим среднее
    mean = round(values.mean(), 2)
    # Вычислим 2,5% квантиль
    lower = round(values.quantile(KVANT25), 2)
     # Вычислим 97,5% квантиль
    upper = round(values.quantile(1-KVANT25), 2)
    # Средний показатель по реальным данным
    target_mean = target.mean()
    # Средний показатель по предсказанным данным
    pred_mean = predictions.mean()

    risk_of_loss = np.mean(values < 0)*100
    
    # Построим гистограмму распределения прибыли при применении метода Bootstrap
    plt.figure(figsize=(16,8))
    plt.hist(values, 50, color='lightgreen')
    plt.plot([mean,mean], [0, 65], 'k-', lw=4, color='green')
    plt.plot([lower,lower],[0, 65], 'k-', lw=2, color='red')
    plt.plot([upper,upper],[0, 65], 'k-', lw=2, color='red')
    plt.legend(['Среднее значение', '95% доверительный интервал'], fontsize=12)
    plt.xlabel('Прибыль, млн.руб', fontsize=15)
    plt.ylabel('Частота', fontsize=15)
    plt.title('Гистрограмма распределения прибыли при применении метода Bootstrap', fontsize=15)
    plt.show()
        
    return    mean, lower, upper, risk_of_loss, target_mean, pred_mean

In [22]:
# Применим технику Bootstrep для регионов
# Первый регион
cash0, kvant0, kvantm0, risk0, targ0, pred0 = analysis(target_valid_0, predictions_valid_0)
# Второй регион
cash1, kvant1, kvantm1, risk1, targ1, pred1 = analysis(target_valid_1, predictions_valid_1)
# Третий регион
cash2, kvant2, kvantm2, risk2, targ2, pred2 = analysis(target_valid_2, predictions_valid_2)

# Сформируем таблицу для подсчтеа прибыли к прогнозу модели по регионам
data = {'Прибыль, руб.':[cash0, cash1, cash2], '2,5%-квантиль:':[kvant0, kvant1, kvant2], 
        '97,5%-квантиль:':[kvantm0, kvantm1, kvantm2], 'Риск':[risk0, risk1, risk2],
       'Среднее по реальным данным':[targ0, targ1, targ2],
       'Среднее по предсказанным данным':[pred0, pred1, pred2]} 
selections = pd.DataFrame(data, index =['Первый регион', 'Второй регион', 'Третий регион']) 
display(selections)

,"Прибыль, руб.","2,5%-квантиль:","97,5%-квантиль:",Риск,Среднее по реальным данным,Среднее по предсказанным данным
Первый регион,403.02,-127.78,919.88,6.5,92.078597,92.592568
Второй регион,515.92,100.90,917.96,0.5,68.723136,68.728547
Третий регион,413.93,-120.73,998.09,7.4,94.884233,94.965046


## Выводы

Проведено исследование, по результатам которого можно сделать следующие выводы:
- Наиболее точно модель линейной регрессии отработала на втором регионе, что подтверждается как самым низким показателем RMSE из всех трех моделей, так и неразличающиеся вплоть до тысячных средние по предсказанным и реальным данным
- Вычислена точка безубыточности, которая составила 111,11 тысяч баррелей на одну скважину. Ни один из трех регионов не имеет таких показателей
- После применения техники Bootstrap для оценки распределения прибыли можно сделать вывод, что наиболее перспективный регион для разработки - второй. Это подтверждается как самым высоким показателем предсказанной прибыли, так и минимальным показателем риска 0,3 %. Первый и третий регион имеют очень высокие показатели риска - соответственно 6 и 6,2 процента.